<a href="https://colab.research.google.com/github/SowmyaaKarthik/Twitter-DataSet-Using-Snscrape-Streamlit/blob/main/Twitter_Data_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 2.9 MB/s eta 0:00:00


In [2]:
!pip install langcodes[data]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00


In [3]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
from langcodes import * #for finding the language name from language codes
import pandas as pd #for creating the dataframe
import datetime #for finding the current timestamp
import pytz
import snscrape.modules.twitter as sntwitter #scrapping the twitter data

In [10]:
%%writefile function_file2.py



from langcodes import * 
import pandas as pd 
import datetime 
from datetime import timedelta
import snscrape.modules.twitter as sntwitter
import json


def scrap_datas(tag, from_date, to_date, lim):
  query = f"{tag} since:{from_date} until:{to_date}"
  limit = lim
  d_tweets = []
  scraper = sntwitter.TwitterSearchScraper(query)
  for tweet in scraper.get_items():
    if len(d_tweets) == lim:
      break
    else:
      d_tweets.append([tweet.id,tweet.url,tweet.date,tweet.user.username,tweet.content,tweet.replyCount,tweet.retweetCount,tweet.lang,tweet.source, tweet.likeCount])

  language_dict = {"Unknown language [qst]" : "Twit-short txt",
                   "Unknown language [qme]" : "Twit-media link",
                   "Unknown language [qam]" : "Twit-mentions only", 
                   "Unknown language [qct]" : "Twit-cashtags", 
                   "Unknown language [qht]" : "Twit-hashtags",
                   "Unknown language" : "Undefined language"}

  for i in d_tweets:
    i[7] = Language.make(language=i[7]).display_name()

  for i in d_tweets:
    if i[7] in language_dict.keys():
      i[7] = language_dict[i[7]]
    else:
      i[7] = i[7]

  da = pd.DataFrame(d_tweets, columns = ["Id", "URL", "Date posted", "User Name", "Content", "Reply count", "Retweet count", "Language", "Source" , "Like count"])
  da["Source"] = da["Source"].apply(lambda x:x.split("=")[1].strip("rel"))
  da["Id"] = da["Id"].astype("str")
  return da

def update_data(tag, df_name):
  df_name["Date posted"] = df_name["Date posted"].apply(lambda x:(str(x)).split("+")[0])
  #Creating document of the fetched data
  data_list = []
  for i in range(df_name.shape[0]):
    x = pd.DataFrame.to_json(df_name.iloc[i, :])
    data_list.append(json.loads(x))

  #connecting to the server
  import pymongo
  client = pymongo.MongoClient("mongodb://localhost:27017")
  db = client.Twitter_data
  records = db.Twitter
  #inserting document into the database
  x = datetime.datetime.now()+timedelta(minutes = 330)
  cur_tsr = str(x).split(".")[0]
  htag = tag+"_"+cur_tsr
  daata = {htag : data_list}
  info = records.insert_one(daata)
  if info != 0:
    return "Inserted succcessfully"
  elif info == 0:
    return "Something went wrong please try again"

Overwriting function_file2.py


In [13]:
%%writefile streamlit_app.py
import json
from function_file2 import  scrap_datas, update_data
import streamlit as st


st.markdown("TWEET SCRAPER", unsafe_allow_html=True)
hashtag = st.text_input('Enter your hashtag for search', '#')
f_date = st.date_input('Enter your Start Date')
from_date = str(f_date).replace("/", "-")
t_date = st.date_input('Enter your End Date')
to_date = str(t_date).replace("/", "-")
limit = st.number_input('Set Limit', 10)
if st.button("Display Data"):
  df = scrap_datas(hashtag, from_date, to_date, limit)
  st.table(df)
if st.button("Download Data"):
  df = scrap_datas(hashtag, from_date, to_date, limit)
  df_c = df.copy()
  df_c["Id"] = df_c["Id"].astype("str")
  df_csv = df_c.to_csv()
  st.download_button("Download as CSV",data = df_csv,file_name = f"{hashtag}.csv",mime='text/csv')
  st.balloons()
  df_json = df.to_json()
  st.download_button("Download as JSON", data = df_json, file_name = f"{hashtag}.json", mime="application/json")
  st.balloons()

if st.button("Upload data to the database"):
  df = scrap_datas(hashtag, from_date, to_date, limit)
  message = update_data(hashtag, df)
  st.write(message)
  st.balloons()

Writing streamlit_app.py


In [ ]:
!streamlit run /content/streamlit_app.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session f0f9cbd73dcad29


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.74.109.212:8501

npx: installed 22 in 5.541s
your url is: https://metal-clubs-poke-34-74-109-212.loca.lt
